![임도형 커멘트](https://github.com/dhrim/keras_example_seminia_2020/raw/master/comment.png)

# 개요

- 원 본 : https://keras.io/examples/vision/knowledge_distillation/
- 작업 : 영상 분류
- 데이터 : MNIST
- 적용 모델 : CNN


<br>


# 지식 전수, Knowledge Distillation

학습한 선생님 모델이 학생 모델에게 지식을 전수하는 것.

보통 학생 모델은 그 크기가 작다. 모델 압축이기도 하다.

<br>

참고 문서 : https://light-tree.tistory.com/196


<br>


# 핵심 사항

학습에 사용되는 label(0, 1, 0)에 곧이 곧대로 맞출 것이 아니라.

이미 학습한 teacher의 softmax 출력(0.1, 0.7, 0.2) 데로 맞추면 될 것이다.

미리 학습하여 결과를 보이는 teacher의 출력을 낸다면 문제 없지 않을까.

```
loss =   0.1 * loss(label, studnet_output) + 0.9 * loss(softmax(teacher_prediction), softmax(student_prediction))
```

```
label = ( 0, 1, 0 )
student_prediction = ( 0.1, 0.7, 0.2 )
teacher_prediction = ( 0.1, 0.7, 0.2 )
```


```




# 태그
```
#knowledge_distillation
#model_compression
```

# Knowledge Distillation

**Author:** [Kenneth Borup](https://twitter.com/Kennethborup)<br>
**Date created:** 2020/09/01<br>
**Last modified:** 2020/09/01<br>
**Description:** Implementation of classical Knowledge Distillation.

## Introduction to Knowledge Distillation

Knowledge Distillation is a procedure for model
compression, in which a small (student) model is trained to match a large pre-trained
(teacher) model. Knowledge is transferred from the teacher model to the student
by minimizing a loss function, aimed at matching softened teacher logits as well as
ground-truth labels.

The logits are softened by applying a "temperature" scaling function in the softmax,
effectively smoothing out the probability distribution and revealing
inter-class relationships learned by the teacher.

**Reference:**

- [Hinton et al. (2015)](https://arxiv.org/abs/1503.02531)

## Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


## Construct `Distiller()` class

The custom `Distiller()` class, overrides the `Model` methods `train_step`, `test_step`,
and `compile()`. In order to use the distiller, we need:

- A trained teacher model
- A student model to train
- A student loss function on the difference between student predictions and ground-truth
- A distillation loss function, along with a `temperature`, on the difference between the
soft student predictions and the soft teacher labels
- An `alpha` factor to weight the student and distillation loss
- An optimizer for the student and (optional) metrics to evaluate performance

In the `train_step` method, we perform a forward pass of both the teacher and student,
calculate the loss with weighting of the `student_loss` and `distillation_loss` by `alpha` and
`1 - alpha`, respectively, and perform the backward pass. Note: only the student weights are updated,
and therefore we only calculate the gradients for the student weights.

In the `test_step` method, we evaluate the student model on the provided dataset.

In [ ]:

class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = student
        self.student = teacher

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)


            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


## Create student and teacher models

Initialy, we create a teacher model and a smaller student model. Both models are
convolutional neural networks and created using `Sequential()`,
but could be any Keras model.

In [ ]:
# Create the teacher
teacher = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(512, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="teacher",
)

# Create the student
student = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(16, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="student",
)

# Clone student for later comparison
student_scratch = keras.models.clone_model(student)

![임도형 커멘트](https://github.com/dhrim/keras_example_seminia_2020/raw/master/comment.png)


student 모델은 teacher 모델에 비해 70배 작다.

In [ ]:
teacher.summary()
student.summary()

Model: "teacher"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 256)       2560      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 14, 14, 256)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 512)         1180160   
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                250890    
Total params: 1,433,610
Trainable params: 1,433,610
Non-trainable params: 0
_________________________________________________

## Prepare the dataset

The dataset used for training the teacher and distilling the teacher is
[MNIST](https://keras.io/api/datasets/mnist/), and the procedure would be equivalent for any other
dataset, e.g. [CIFAR-10](https://keras.io/api/datasets/cifar10/), with a suitable choice
of models. Both the student and teacher are trained on the training set and evaluated on
the test set.

In [ ]:
# Prepare the train and test dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize data
x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 28, 28, 1))


11493376/11490434 [==============================] - 0s 0us/step


## Train the teacher

In knowledge distillation we assume that the teacher is trained and fixed. Thus, we start
by training the teacher model on the training set in the usual way.

In [ ]:
# Train teacher as usual
teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate teacher on data.
teacher.fit(x_train, y_train, epochs=5)
teacher.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1430 - sparse_categorical_accuracy: 0.9565
Epoch 2/5
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0935 - sparse_categorical_accuracy: 0.9730
Epoch 3/5
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0819 - sparse_categorical_accuracy: 0.9763
Epoch 4/5
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0755 - sparse_categorical_accuracy: 0.9787
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 0.1361 - sparse_categorical_accuracy: 0.9677


[0.13612306118011475, 0.9677000045776367]

## Distill teacher to student

We have already trained the teacher model, and we only need to initialize a
`Distiller(student, teacher)` instance, `compile()` it with the desired losses,
hyperparameters and optimizer, and distill the teacher to the student.

In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=3)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

Epoch 1/3
1875/1875 [==============================] - 18s 10ms/step - sparse_categorical_accuracy: 0.9818 - student_loss: 0.1534 - distillation_loss: 0.0222
Epoch 2/3
1875/1875 [==============================] - 18s 10ms/step - sparse_categorical_accuracy: 0.9855 - student_loss: 0.1382 - distillation_loss: 0.0184
Epoch 3/3
313/313 [==============================] - 1s 4ms/step - sparse_categorical_accuracy: 0.9816 - student_loss: 0.1484


0.9815999865531921

## Train student from scratch for comparison

We can also train an equivalent student model from scratch without the teacher, in order
to evaluate the performance gain obtained by knowledge distillation.

In [ ]:
# Train student as doen usually
student_scratch.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch.fit(x_train, y_train, epochs=3)
student_scratch.evaluate(x_test, y_test)

Epoch 1/3
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2313 - sparse_categorical_accuracy: 0.9314
Epoch 2/3
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0901 - sparse_categorical_accuracy: 0.9721
Epoch 3/3
313/313 [==============================] - 1s 2ms/step - loss: 0.0586 - sparse_categorical_accuracy: 0.9794


[0.058573201298713684, 0.9793999791145325]

![임도형 커멘트](https://github.com/dhrim/keras_example_seminia_2020/raw/master/comment.png)

- teacher model : 3 epoch loss = 0.97
- teacher model : 5 epoch loss = 0.98
- student model from scratch : 3 epoch loss = 0.978
- student model by distilation : 3 epoch loss = 0.98

If the teacher is trained for 5 full epochs and the student is distilled on this teacher
for 3 full epochs, you should in this example experience a performance boost compared to
training the same student model from scratch, and even compared to the teacher itself.
You should expect the teacher to have accuracy around 97.6%, the student trained from
scratch should be around 97.6%, and the distilled student should be around 98.1%. Remove
or try out different seeds to use different weight initializations.